In [1]:
import os

In [2]:
%pwd

'F:\\Study\\Deep_learning_Project\\Kidney-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'F:\\Study\\Deep_learning_Project\\Kidney-Disease-Classification'

In [21]:
#entitiy
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weight: str
    params_classes: int
    

In [7]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_ymal, create_directories

In [26]:
#configuration
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_ymal(config_filepath)
        self.params = read_ymal(params_filepath)
        create_directories([self.config.artifact_root])

    def get_prepare_base_model_config(self)-> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([config.root_dir])
        return PrepareBaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            params_image_size = self.params.IMAGE_SIZE,
            params_learning_rate = self.params.LEARNING_RATE,
            params_include_top = self.params.INCLUDE_TOP,
            params_weight = self.params.WEIGHTS,
            params_classes = self.params.CLASSES
            
        )
    

In [20]:
#components
import os
from urllib.request import urlopen
from zipfile import ZipFile
import tensorflow as tf

In [32]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape= self.config.params_image_size,
            weights = self.config.params_weight,
            include_top = self.config.params_include_top
        )
        self.save_model(path = self.config.base_model_path, model= self.model)

    @staticmethod
    def _prepare_full_model(model,classes,learning_rate,freeze_all= True, freeze_till=None):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till>0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
        #flatten layer
        flatten_in = tf.keras.layers.Flatten()(model.output)
        #output layer
        prediction = tf.keras.layers.Dense(classes, activation='softmax')(flatten_in)
        #defining the model
        full_model = tf.keras.models.Model(
            inputs = model.input,
            outputs = prediction
        )
        #compiling the model
        full_model.compile(
            optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics = ['accuracy']
        )
        full_model.summary()
        return full_model
        
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model = self.model,
            classes = self.config.params_classes,
            freeze_all = True,
            freeze_till = None,
            learning_rate = self.config.params_learning_rate
        )
        self.save_model(path = self.config.updated_base_model_path, model= self.full_model)
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    
    
            
        

In [33]:
#pipline
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-03-17 13:00:02,104: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-17 13:00:02,109: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-17 13:00:02,112: INFO: common: created directory at: artifacts]
[2024-03-17 13:00:02,114: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-03-17 13:00:02,488: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 244, 244, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 244, 244, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 244, 244, 